<a href="https://colab.research.google.com/github/sachins301/UTA-Distributed-Computing/blob/main/UTA%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install xmltodict

In [30]:
import requests
import xmltodict
import json
import pytz
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import time
import os
import concurrent.futures
import logging


In [22]:
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:

# Create a custom logger
logger = logging.getLogger(__name__)
logger.propagate = False # do not pass logs to the default logger

# Create handlers
c_handler = logging.StreamHandler()
f_handler = logging.FileHandler('/content/drive/My Drive/Colab Notebooks/utaproject.log', mode='w')
c_handler.setLevel(logging.WARNING)
f_handler.setLevel(logging.ERROR)

# Create formatters and add it to handlers
c_format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(c_handler)
logger.addHandler(f_handler)
logger.info("Starting ", __name__)

In [18]:
def retrieve_data(route_id, onward_calls = False):
  token = userdata.get('TOKEN')
  url = f"http://api.rideuta.com/SIRI/SIRI.svc/VehicleMonitor/ByRoute?route={route_id}&onwardcalls={onward_calls}&usertoken={token}"
  response = requests.get(url)
  if response.status_code != 200:
    return None
  xml_data = xmltodict.parse(response.text)
  json_data = json.dumps(xml_data)
  return json_data

In [19]:
def convert_to_epoch(timestamp):
    date_part, rest = timestamp.split('.')
    microseconds_part = rest[:-6]  # Extract microseconds part
    timezone_part = rest[-6:]      # Extract timezone part
    microseconds_part = microseconds_part[:6]  # Take only the first 6 digits for microseconds
    timestamp = date_part + '.' + microseconds_part + timezone_part
    dt = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%f%z")
    epoch_time = int(dt.timestamp())

    return str(epoch_time)

In [20]:
def save_data(route_id, json_data):
  file_name = route_id + convert_to_epoch(xml_data["Siri"]["ResponseTimestamp"])  + ".json"
  with open('/content/drive/My Drive/Colab Notebooks/'+file_name, 'w', encoding='utf-8') as f:
    f.write(json_data)

In [28]:
route_id_list = ['4', '455', '1', '2', '220']
def process_route(route_id):
    while True:
        json_data = retrieve_data(route_id)
        if json_data:
            save_data(route_id, json_data)
        time.sleep(5)

In [29]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(process_route, route_id_list)

KeyboardInterrupt: 

In [27]:
# No of default workers /cpu available for multithreading
print(os.cpu_count() + 4)

6


In [14]:
# Opening JSON file
f = open('/content/drive/My Drive/Colab Notebooks/'+file_name)
data = json.load(f)